## Строим вопрос-ответного бота по технологии Retrieval-Augmented Generation на LangChain

[LangChain](https://python.langchain.com) - это набирающая популярность библиотека для работы с большими языковыми моделями и для построения конвейеров обработки текстовых данных. В одной библиотеке присутствуют все элементы, которые помогут нам создать вопрос-ответного бота на наборе текстовых данных: вычисление эмбеддингов, запуск больших языковых моделей для генерации текста, поиск по ключу в векторных базах данных и т.д.

Для начала, установим `langchain` и сопутствующие библиотеки.

In [1]:
%pip install -q langchain sentence_transformers lancedb unstructured yandex_chain yandexcloud


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


## Разбиваем документ на части

Для работы retrival augmented generation нам необходимо по запросу найти наиболее релевантные фрагменты исходного текста, на основе которых будет формироваться ответ. Для этого нам надо разбить текст на такие фрагменты, по которым мы сможем вычислять эмбеддинг, и которые будут с запасом помещаться во входное окно используемой большой языковой модели.

Для этого можно использовать механизмы фреймворка LangChain - например, `RecursiveCharacterTextSplitter`. Он разбивает текст на перекрывающиеся фрагменты по набору типовых разделителей - абзацы, переводы строк, разделители слов.

> **ВАЖНО**: Перед выполнением ячейки не забудьте установить имя пользователя, которое вы использовали на предыдущем шаге.

Размер `chunk_size` нужно выбирать исходя из нескольких показателей:
* Допустимая длина контекста для эмбеддинг-модели. Yandex GPT Embeddings допускают 2048 токенов, в то время как многие открытые модели HuggingFace имеют длину контекста 512-1024 токена
* Допустимый размер окна контекста большой языковой модели. Если мы хотим использовать в запросе top 3 результатов поиска, то 3 * chunk_size+prompt_size+response_size должно не превышать длины контекста модели.

In [2]:
import langchain
import langchain.document_loaders

user = 'shwars'
chunk_size = 1024
chunk_overlap=50
source_dir = f"/home/jupyter/datasphere/s3/s3store/{user}/text"

loader = langchain.document_loaders.DirectoryLoader(source_dir,glob="*.txt",show_progress=True,recursive=True)
splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
fragments = splitter.create_documents([ x.page_content for x in loader.load()])
len(fragments)

100%|██████████| 3/3 [00:02<00:00,  1.02it/s]


65

## Вычисляем эмбеддинги для всех фрагментов

Для вычисления эмбеддингов можно взять какую-нибудь модель из репозитория HuggingFace, с поддержкой русского языка. LangChain содержит свои встроенные средства работы с эмбеддингами, и поддерживает модели из HuggingFace: 

In [3]:
import langchain.embeddings
embeddings = langchain.embeddings.HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v1")
sample_vec = embeddings.embed_query("Hello, world!")
len(sample_vec)

512

Также можно использовать более продвинутую модель [эмбеддингов от Yandex GPT](https://cloud.yandex.ru/docs/yandexgpt/api-ref/Embeddings/). Её можно использовать через библиотеку `yandex_chain`. Не забудьте при необходимости исправить параметр `folder_id` в соответствии с вашими данными.

In [4]:
import os
from yandex_chain import YandexEmbeddings
api_key = os.environ['api_key']
folder_id = "b1g6krtrd2vcbunvjpg6"
embeddings = YandexEmbeddings(folder_id=folder_id,api_key=api_key)
res = embeddings.embed_documents(['Hello','there'])
len(res),len(res[0])

(2, 256)

## Cохраняем эмбеддинги  в векторную БД

Для поиска нам нужно уметь быстро сопоставлять эмбеддинг запроса, и эмбеддинги всех фрагементов наших исходных материалов. Для этого используются векторные базы данных. Для крупных проектов имеет смысл использовать серьезные инструменты, типа [OpenSearch](https://opensearch.org/) (доступный [в виде сервиса в облаке Yandex Cloud](https://cloud.yandex.ru/services/managed-opensearch)), но для нашего примера мы используем небольшую векторную БД [LanceDB](https://lancedb.com/), хранящую индекс в директории на диске.

Первоначально создадим базу данных, добавив туда одну строчку:

In [6]:
from langchain.vectorstores import LanceDB
import lancedb
import os

db_dir = "store"

os.makedirs(db_dir,exist_ok=True)

db = lancedb.connect(db_dir)
table = db.create_table(
    "vector_index",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

Далее проиндексируем все выделенные ранее фрагменты нашей документации, используя реализованный нами выше адаптер для YandexGPT-эмбеддингов. 

> В зависимости от объема текста, эта ячейка может выполняться достаточно длительное время - вспомним про задержку в 1 сек между запросами!

In [7]:
db = LanceDB.from_documents(fragments, embeddings, connection=table)

Теперь посмотрим, насколько хорошо находятся фрагменты текста по какому-то запросу:

In [9]:
q="Почему стоит покупать iPhone?"

res = db.similarity_search(q)
for x in res:
    print('-'*40)
    print(x.page_content)

----------------------------------------
genexy defall five и пойти потому что никому не интересно он ничем не отличается от прошлого это новый айфон все спрашивают про новый айфон вы знаете что меня часто узнают на улице мне никогда не спрашивают никакого другого телефона я с разными хожу но когда видят этот телефон все говорят о это что новый айфон вам он только вышел классно классно а покажи да и пощупать я натурально подхожу к баристе он говорит дай потрогать пожалуйста потому что это айфон всем интересно как что зачем и почему что еще мы хотим обсудить фишки
----------------------------------------
менее давайте сравним эти 2 действительно крутых флагмана посмотрим как они между собой отличаются потому что есть о чем поговорить 10 пунктов о самых крутых смартфонах прямо сейчас поехали У меня Начнем с комплекта поставки собственно распаковку айфона и 1 эмоции вы можете посмотреть в соответствующем ролике у нас на канале ну а комплект айфона представляет из себя телефон немножко мак

Ещё один полезный интерфейс для поиска текстов - это `Retriever`, убедимся, что он тоже работает:

In [19]:
retriever = db.as_retriever(
    search_kwargs={"k": 5}
)
res = retriever.get_relevant_documents(q)
for x in res:
    print(x.page_content)

Народ всем привет я достаю из кармана айфон 14 про макс но вы же думали что я пятнашку добуду я конечно фокусник знатный но эта магия сейчас под запретом так вот что я вам хочу сказать уже совсем скоро 12 сентября ребята в купертино порадуют нас новым айфон 15 расскажут что там прикольного и главное вообще самое масштабное изменение это переход на ю эс би си в базовых айфон 15 15 + это будет медленный юсб 2 0 по стандарту юсбс а в прошках мы ожидаем что то очень быстрое также usb c конечно же а это значит огромное количество аксессуаров будет изменено беттери пати всякие наушники и прочее прочее что сейчас заряжается по лайтнингу будет заряжаться по usb c поэтому я вам рекомендую уже сейчас озаботиться заменой с 1 стороны конечно же можно будет купить обновленные airpods от apple с другой стороны а нахрена это нужно потому что понеслась засылай дядя сережа давай быстрее в этой покраске Вот такой вот airpods следом идет зарядка оригинальная которой в комплекте нету эйрподсы конечно же п

## Подключаемся к Yandex GPT

Фреймворк LangChain поддерживает интеграцию с различными большими языковыми моделями, включая Yandex GPT.Попросим модель ответить на вопрос от лица сотрудника магазина сотовой связи:

In [11]:
from langchain.llms import YandexGPT
instructions = """
Представь себе, что ты консультант магазина сотовой связи.
Твоя задача - вежливо и по мере своих сил отвечать на все вопросы собеседника."""

llm = YandexGPT(api_key=api_key,
                instruction_text = instructions)
res = llm(q)
print(res)

iPhone - это отличный выбор для тех, кто ищет современный смартфон с множеством функций. Вот несколько причин, почему стоит купить iPhone:

1. Дизайн: iPhone имеет стильный дизайн, который выглядит современно и привлекательно. Он также доступен в различных цветах и материалах, что позволяет каждому найти идеальный вариант для себя.

2. Производительность: iPhone оснащен мощным процессором и большим объемом оперативной памяти, что обеспечивает быструю и плавную работу приложений и игр.

3. Камера: iPhone имеет отличную камеру, которая позволяет делать качественные фотографии и видео. Она также поддерживает функцию Live Photos, которая создает анимированные снимки.

4. Совместимость: iPhone работает на операционной системе iOS, которая известна своей стабильностью и безопасностью. Это означает, что вы можете быть уверены в том, что ваш телефон будет работать без сбоев и уязвимостей.

5. Приложения и сервисы: Apple предлагает широкий спектр приложений и сервисов, таких как Apple Music, Ap

В данном примере мы пока что никак не использовали наши текстовые документы.

## LangChain Prompts

Для работы с промптами в LangChain существует специальный механизм шаблонов.

In [15]:
prompt_text = """
Ответь на следующий вопрос пользователя по имени {name},
который приводится ниже в тройных кавычках, обращаясь к нему 
вежливо и по имени. Вот сам вопрос:
```{question}```
"""

prompt=langchain.prompts.PromptTemplate(template=prompt_text,input_variables=['name','question'])

res = llm(prompt.format(name='Вася', question='Какой макбук купить?'))
print(res)

'Уважаемый Вася, я понимаю, что выбор ноутбука непростая задача. Макбуки очень популярны, но важно учесть ваш бюджет, задачи, которые вы будете решать с ноутбуком, а также личные предпочтения. Предлагаю вам рассмотреть несколько вариантов:\n\n1. MacBook Air - компактный и лёгкий ноутбук с процессором Intel. Он идеален для работы с офисными программами и интернет-сёрфингом.\n\n2. MacBook Pro 13" - отличный выбор для тех, кто часто путешествует или работает в дороге. Это ноутбук с хорошей производительностью и временем автономной работы.\n\n3. MacBook Pro 16" - если вам нужен более мощный ноутбук для работы с видеомонтажом, графикой или программированием, то этот вариант для вас.\n\n4. Если вы предпочитаете Apple Silicon, то можно рассмотреть MacBook Air M1 или MacBook Pro M1. Они работают на новом процессоре M1, который обеспечивает высокую производительность и долгое время автономной работы.\n\nНадеюсь, эти варианты помогут вам определиться с выбором ноутбука. Если у вас возникнут допо

## Собираем Retrieval-Augmented Generation

Пришла пора соединить всё вместе и научить бота отвечать на вопросы, подглядывая в наш текстовый корпус. Для этого используем механизм цепочек (*chains*). Основную функциональность реализует `StuffDocumentsChain`, которая делает следующее:

1. Берёт коллекцию документов `input_documents`
1. Каждый из них пропускает через `document_prompt`, и затем объединяет вместе.
1. Данный текст помещается в переменную `document_variable_name`, и передаётся большой языковой модели `llm`

В нашем случае `document_prompt` не будет модицицировать документ, а основной запрос к LLM будет содержать в себе инструкции для Yandex GPT. 

In [23]:
import langchain.chains 

# Промпт для обработки документов
document_prompt = langchain.prompts.PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)

# Промпт для языковой модели
document_variable_name = "context"
stuff_prompt_override = """
Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
Текст:
-----
{context}
-----
Вопрос:
{query}"""
prompt = langchain.prompts.PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Создаём цепочку
llm_chain = langchain.chains.LLMChain(llm=llm, prompt=prompt)
chain = langchain.chains.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)

# Делаем запрос к Vector DB
q = 'Какой макбук купить студенту, который учится на первом курсе технического вуза?'
res = retriever.get_relevant_documents(q)

chain.run(input_documents=res, query=q)

'Ответ:\nВ тексте нет информации о том, какой макбук можно порекомендовать студенту-первокурснику технического вуза. Автор скорее имеет в виду, что в качестве первого компьютера для учебы можно рассмотреть и ноутбуки других производителей, например, Asus, Acer, HP или Lenovo.'

Чтобы ещё более улучшить результат, мы можем использовать хитрый механизм перемешивания документов, таким образом, чтобы наиболее значимые документы были ближе к началу запроса. Также мы оформим все операции в одну функцию `answer`:

In [24]:
from langchain.document_transformers import LongContextReorder
reorderer = LongContextReorder()

def answer(query,reorder=True,print_results=False):
  results = retriever.get_relevant_documents(query)
  if print_results:
        for x in results:
            print(f"{x.page_content}\n--------")
  if reorder:
    results = reorderer.transform_documents(results)
  return chain.run(input_documents=results, query=query)

In [26]:
answer("Какие наушики лучше купить чтобы слушать рок?",print_results=True)

достаточно дорого поэтому Я приготовил для вас этот женщина Альтернативу которая по моему мнению вполне годится для того чтобы кайфануть от звука не под брендом apple понеслась airpods базовые я предлагаю заменить обновленными нафтинг игр базовыми опять же да то есть у них есть версия с затычками шумодавами бла бла бла Здесь же это очень прикольные наушники если вы вдруг не видели давайте покажу обратите внимание как настенька открывается забавно вы просто уничтожаете коробку вот таким вот образом Восстановить ее уже не получится и это прикол но самое в том что вот этот забавный дизайн это не киоск Это короче аналог эйрподс про но я на самом деле имел в виду другие наушники которые больше подходят как замена базы airpods потому что имеет такой же стиль размещений без резиночек Нафин стиг вот эти я имел в виду или я ими пользуюсь и говоришь не хуже чем базовые эйрподсы ну Мы ему поверим да потому что да на самом деле базы аирподсы на мой взгляд для прослушивания музыки годятся весьма ну

'Если вы любите слушать рок, то вам лучше купить наушники с хорошим басом и чистыми высокими частотами. Также обратите внимание на удобство наушников и качество микрофона.'

Можно сравнить результаты, выдаваемые Yandex GPT напрямую, с результатами нашей генерации на основе документов:

In [28]:
def compare(q):
    print(f"Ответ Yandex GPT: {llm(q)}")
    print(f"Ответ бота: {answer(q)}")
    
compare("Какой смартфон лучше купить фотографу?")

Ответ Yandex GPT: Фотографу, который хочет получить высокое качество снимков, рекомендуется купить смартфон с хорошей камерой. Также следует обратить внимание на следующие характеристики:

1. Разрешение камеры - не менее 24 Мп.
2. Оптический зум - чем больше, тем лучше.
3. Стабилизация изображения - поможет избежать смазанных снимков при съемке с рук.
4. Наличие встроенного искусственного интеллекта (AI) - позволит делать более качественные снимки.
5. Большой объем оперативной и встроенной памяти - для хранения фотографий и быстрого запуска приложений.
6. Поддержка быстрой зарядки - поможет быстро зарядить аккумулятор.
7. Хорошая производительность - для быстрой обработки снимков и работы с большим количеством файлов.
8. Защищенный корпус - для защиты от воды и пыли.
9. Беспроводные интерфейсы - для подключения к компьютеру и передачи файлов.
10. Хорошая цена - чтобы не переплатить за ненужные функции.
Ответ бота: Из текста следует, что для фотографера лучше выбрать смартфон с более ка

## Экспериментируем

Мы написали основной код, но при этом качество ответов всё ещё далеко от идеальных. Рекомендуем вам поэкспериментировать самостоятельно, чтобы добиться от чат-бота более качественного ответа:

1. Попробуйте менять температуру генерации текста, установив `llm.temperature=0.01` (пробуйте значения от 0.001 до 1)
2. Посмотрите на тексты, которые находятся в результате поиска, и если релевантных тексту фрагментов нет - попробуйте расширить объем входной базы знаний, или дописать в неё какие-то тексты самостоятельно.
3. Добавьте к боту немного читчата (фразы типа *привет, как дела*, *а кто ты такой* и т.д.), поместив соттветствующие вопрос-ответные пары в базу знаний. 
4. Попробуйте экспериментировать с параметром `reorder`
5. Попробуйте изменять промпты, и просить модель по разному вести себя в зависимости от того, содержится ли ответ в имеющихся текстах. Например, Вы можете попросить модель саму порассуждать, если ответа на вопрос в явноми виде в тексте не содержится.

## Сохраняем векторную БД в Storage

Для следующего этапа - вопрос-ответного бота - нам потребуется созданная нами база данных с документами. Поэтому скопируем её на хранилище s3:

In [29]:
!cp -R ./store /home/jupyter/datasphere/s3/s3store/shwars/